In [ ]:
# this script is made for processing metadata files to those of same header

In [68]:
model <- read.csv("sample_to_run_info.csv", header = TRUE) # 模板表格

In [69]:
ori_origin_metadata <- read.csv("metadata.csv", header = TRUE) # 待转换的表格

In [14]:
# dplyr 包似乎比较理想
library(dplyr)

In [ ]:
##################
# 这一行代码复制粘贴这用，里面的内容需要调整，因为select不能识别不存在的列
select(tibble_origin_metadata, checking , project_id , our_project_id , sample_name , original_sample_description , 
       curated_sample_description , run_id , sample_id , second_sample_id , experment_type , nr_reads_sequenced , 
       instrument_model , disease , phenotype , is_disease_stage_available , disease_stage , more , more_info , contry , 
       collection_date , sex , host_age , diet , longitude , latitude , BMI , recent_antibiotics_use , antibiotics_used , 
       antibiotics_does , days_without_antibiotics_use)
##################

In [70]:
# 首先要对metadata进行处理，包括更改列名以及取子集

tibble_origin_metadata <- tbl_df(ori_origin_metadata)
tibble_origin_metadata[1,]


Run,Assay.Type,AvgSpotLen,Bases,BioProject,BioSample,BioSampleModel,Bytes,Center.Name,Collection_date,⋯,lat_lon,Library.Name,LibraryLayout,LibrarySelection,LibrarySource,Organism,Platform,ReleaseDate,Sample.Name,SRA.Study
<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
SRR15323178,AMPLICON,364,56844649,PRJNA751473,SAMN20519100,"MIMARKS.survey,MIGS/MIMS/MIMARKS.human-gut",35928021,KASETSART UNIVERSITY,2016-10-01,⋯,13.7323 N 100.5368 E,Atopic-085,PAIRED,PCR,METAGENOMIC,human gut metagenome,ILLUMINA,2021-09-16T00:00:00Z,Atopic-085,SRP330833


In [71]:
###
sec_tibble_selected <- select(tibble_origin_metadata, project_id=BioProject , sample_name=Sample.Name ,  
       run_id=Run , sample_id=SRA.Study , second_sample_id=BioSample , experment_type=Assay.Type)
out_metadata <- bind_rows(model, sec_tibble_selected) %>%
    filter(project_id == "PRJNA751473") # 过滤掉模板文件里原本带的那几行。
out_metadata[1:3,]

,checking,project_id,our_project_id,sample_name,original_sample_description,curated_sample_description,run_id,sample_id,second_sample_id,experment_type,⋯,sex,host_age,diet,longitude,latitude,BMI,recent_antibiotics_use,antibiotics_used,antibiotics_does,days_without_antibiotics_use
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,NA,PRJNA751473,NA,Atopic-085,NA,NA,SRR15323178,SRP330833,SAMN20519100,AMPLICON,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,NA,PRJNA751473,NA,Atopic-079,NA,NA,SRR15323179,SRP330833,SAMN20519099,AMPLICON,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,NA,PRJNA751473,NA,Atopic-173,NA,NA,SRR15323180,SRP330833,SAMN20519101,AMPLICON,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [72]:
# 之后是否还可以批量添加信息？比如我现在的phenotype是NA，是否可以批量的添加表型信息？
# 需要的不是join函数
# 可以按run id排序后直接整列修改内容
ori_attach <- read.table("meta", header = TRUE)
attach <- tbl_df(ori_attach)
attach[1,]
sec_selected_attach <- select(attach, run_id=sampleid, phenotype=label)
sec_selected_attach[1:3,]
out_metadata["phenotype"] <- sec_selected_attach["phenotype"]
write.csv(out_metadata, file="curated_metadata.csv", row.names = FALSE)


sampleid,phenotype,label
<chr>,<chr>,<chr>
SRR15323178,Atopic-085,atopic


run_id,phenotype
<chr>,<chr>
SRR15323178,atopic
SRR15323179,atopic
SRR15323180,atopic
